In [1]:
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib
import geopandas as gpd



In [2]:
%load_ext autoreload
%autoreload 2
''
config = RClib.loadconfigs('.\config_gareus.json')
scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = config['baseimageformat'])
#scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
#rcsettings_template = pd.read_csv('.\\allRCsettings_template.csv', sep=';', header=0) 
#print(rcsettings_template)



In [3]:
%load_ext autoreload
%autoreload 2
for index,scan in scandf.iterrows():
    #print(scan.keys())
    RClib.read_rcorthobox(scan)
    #print(scan['orthobox'])
    scan['orthobox'].to_file(config['workspace'] + '/Uruk_Gareus_orthoboxes_utm38.shp', driver='ESRI Shapefile')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.0 0.0 -116.700000989596
POLYGON ((561040.4797042324 3464890.427222059, 561042.3654549695 3464891.375655913, 561052.665812647 3464870.895675162, 561050.7800619099 3464869.947241308, 561040.4797042324 3464890.427222059))
0.0 0.0 -116.700000989596
POLYGON ((561040.4797042324 3464890.427222059, 561042.3654549695 3464891.375655913, 561052.665812647 3464870.895675162, 561050.7800619099 3464869.947241308, 561040.4797042324 3464890.427222059))
6.04419046226541e-09 9.63567725830371e-08 -115.999997947561
POLYGON ((561037.4548454713 3464882.803362839, 561038.428717643 3464883.278351991, 561044.5659255004 3464870.695210001, 561043.5920533286 3464870.220220849, 561037.4548454713 3464882.803362839))
0.0 0.0 -116.000012628985
POLYGON ((561038.3765303404 3464883.252888099, 561040.5905438871 3464884.332735262, 561046.7277473719 3464871.74961042, 561044.5137338252 3464870.669763258, 561038.3765303404 3464883.252888

In [4]:
%load_ext autoreload
%autoreload 2
scandf2= scandf2.apply(RClib.defineResultOutput, foldername='04_GEO', axis=1)
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
scandf3 = pd.DataFrame()
if config['reconstruct']:
    for index,scan in scandf2.iterrows():
        if 'constantrcbox' in config.keys() and Path(config['constantrcbox']).is_file():
            scan['rcboxpath'] = Path(config['constantrcbox'])
        else:
            scan['rcboxpath'] = Path(scan['RCoutputfolder'] / 'base.rcbox')
        scan =RClib.readProcessingstateFile(scan)    
        if RClib.checkProcessingstate(scan,command='align-rccmdpath') and scan['rcboxpath'].is_file():
            if not RClib.checkProcessingstate(scan,command='commandlist_reconstructsettings'):
                scan['commandlist_reconstructsettings']=Path(config['reconstructsettings'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_reconstructsettings')
            if not RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'): 
                scan['commandlist_reconstruct'] = []
                scan['commandlist_reconstruct'].append('-setReconstructionRegion ' + str(scan['rcboxpath']))  
                scan['commandlist_reconstruct'].append('-calculateNormalModel')  
                scan['commandlist_reconstruct'].append('-renameSelectedModel ' + scan['id'] + '_highpoly')    
                scan['commandlist_reconstruct'].append('-save')
                scan['commandlist_reconstruct'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
                print('Starting reconstruction for  ', scan['id'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf3 = scandf3.append(scan)
else:
    scandf3 = scandf2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


NameError: name 'scandf2' is not defined

In [ ]:
scandf4 = pd.DataFrame()
if config['texture']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_texturingsettings'):
                scan['commandlist_texturingsettings']=Path(config['texturingsettings'])
                print('Texturing settings set for ', scan['id'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_texturingsettings')
            if not RClib.checkProcessingstate(scan,command='texturing-rccmdpath'):
                scan['commandlist_texturing'] = []
                scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly') 
                scan['commandlist_texturing'].append('-smooth') 
                scan['commandlist_texturing'].append('-cleanModel')
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_highpoly' + '_smoothed')
                scan['commandlist_texturing'].append('-correctColors geometry') 
                scan['commandlist_texturing'].append('-unwrap')  
                scan['commandlist_texturing'].append('-calculateTexture')
                scan['commandlist_texturing'].append('-simplify 10000000')
                scan['commandlist_texturing'].append('-cleanModel')
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_10M')
                scan['commandlist_texturing'].append('-unwrap')
                scan['commandlist_texturing'].append('-reprojectTexture ' + scan['id'] + '_highpoly' + '_smoothed' + ' ' + scan['id'] + '_10M')
                scan['modelspaths'] = {'modelpath_10M': scan['Resultoutputfolder'] / Path(config['exportname_prefix'] + scan['id'] + '_10M.obj')}
                print(scan['modelspaths'])
                scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_10M ' + str(scan['modelspaths'].get('modelpath_10M')) + ' ' + config['exportsettings1'] )
                scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly' + '_smoothed')   
                scan['commandlist_texturing'].append('-simplify 40000000')
                scan['commandlist_texturing'].append('-cleanModel') 
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_40M')
                scan['commandlist_texturing'].append('-unwrap')
                scan['commandlist_texturing'].append('-reprojectTexture ' + scan['id'] + '_highpoly' + '_smoothed' + ' ' + scan['id'] + '_40M')
                scan['modelspaths']['modelpath_40M'] = scan['Resultoutputfolder'] / Path( config['exportname_prefix'] + scan['id'] + '_40M.obj')
                scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_40M ' + str(scan['modelspaths'].get('modelpath_40M')) + ' ' + config['exportsettings1'] )           
                scan['commandlist_texturing'].append('-save')
                scan['commandlist_texturing'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_texturing', rccmdpathfield='texturing-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='texturing-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf4 = scandf4.append(scan)
else:
    scandf4=scandf3
scandf4.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

Texturing settings set for  PG_boat91_test
{'modelpath_10M': WindowsPath('E:/RCprocessing_test/PG_boat91_test/04_GEO/Zed-ext_0000001_PG_boat91_test_10M.obj')}
E:\RCprocessing_test\PG_boat91_test\03_RC\commandlist_texturing_PG_boat91_test.rccmd


PermissionError: [WinError 32] Der Prozess kann nicht auf die Datei zugreifen, da sie von einem anderen Prozess verwendet wird: 'E:\\RCprocessing_test\\PG_boat91_test\\03_RC\\Processingstate_PG_boat91_test.json'

In [ ]:
if config['orthomosaic']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_orthoimage'):
                if 'constantorthobox' in config.keys() and Path(config['constantorthobox']).is_file():
                    scan['orthoboxpath'] = Path(config['constantorthobox'])
                    scan['commandlist_orthoimage'] = []
                    scan['commandlist_orthoimage'].append('-selectModel ' + scan['id'] + '_highpoly') 
                    scan['commandlist_orthoimage'].append('-calculateOrthoProjection ' + scan['orthoboxpath'] ) 
                    scan['commandlist_orthoimage'].append('-exportOrthoProjection ' + scan['orthoboxpath'] ) 
            

In [ ]:
if config['orthobox2gis']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):